In [3]:
import pandas as pd


def purge():
    import torch
    from numba import cuda
    import gc
    from time import sleep
    try:
        cuda.select_device(0)
        cuda.close()
        cuda.select_device(0)
        gc.collect()
        sleep(2)
        torch.cuda.empty_cache()
    except:
        pass
    
    
checkpoints = [
    # "facebook/wav2vec2-large-slavic-voxpopuli-v2",
    "facebook/wav2vec2-large-960h-lv60-self",
    # "classla/wav2vec2-large-slavic-parlaspeech-hr",
]
optimal_epochs = {
    "facebook/wav2vec2-large-960h-lv60-self": 9,
    "facebook/wav2vec2-large-slavic-voxpopuli-v2": 11,
    "classla/wav2vec2-large-slavic-parlaspeech-hr": 7,  
}
from utils import train_model, eval_model
for checkpoint in checkpoints:
    import os
    os.system("rm -r models/*")
    train_config = dict(
        model_name_or_path = checkpoint,
        TASK = "emotion_optimal_epochs",
        NUM_EPOCH = optimal_epochs.get(checkpoint),
        output_column = "target",
        input_column = "path",
        data_files = {
            "train": "007_train.csv",
            "validation": "007_dev.csv",
        },
        clip_seconds = 10,
    )

    output_dir = train_model(train_config)
    import numpy as np
    for split in "dev test".split():
        results = []
        from pathlib import Path
        found_path = str(list(Path(output_dir).glob("checkpoint-*"))[0])
        print(f"For checkpoint: {checkpoint} a path was found: ", found_path)
        eval_config = dict(
            output_column = "target",
            model_name_or_path= found_path ,
            eval_file= f"007_{split}.csv"
        )

        y_true, y_pred = eval_model(eval_config)
        import datetime
        time = str(datetime.datetime.now())
        results.append(
            {
                **eval_config, 
                **train_config,
                "split": split,
                "y_true": y_true, 
                "y_pred": y_pred,
                "time": time,
                        })
        content = pd.DataFrame(data=results).to_json(None, 
                                           orient="records",
                                           lines=True,
                                           )
        with open("008_results_new_new_new.jsonl", "a") as f:
            f.writelines(content)

Using custom data configuration default-9260442a7fe0355e
Reusing dataset csv (/home/peterr/.cache/huggingface/datasets/csv/default-9260442a7fe0355e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

A classification problem with 5 classes: ['anger', 'fear', 'happiness', 'neutral', 'sadness']


loading configuration file https://huggingface.co/facebook/wav2vec2-large-960h-lv60-self/resolve/main/config.json from cache at /home/peterr/.cache/huggingface/transformers/5681e9346f90f9fc4d72503284e96e6bbdc8bf5a38cafeb6ebf3791120b7570d.e0d02e2ed52b244ae1896cccc2beab5caccc2478b8b3d1131c14666c6e14cfdc
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-large-960h-lv60-self",
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",

The target sampling rate: 16000


...

For checkpoint: facebook/wav2vec2-large-960h-lv60-self a path was found:  models/facebook_wav2vec2-large-960h-lv60-self_emotion_optimal_epochs_/checkpoint-1575


Using custom data configuration default-1cd582fe01726688
Reusing dataset csv (/home/peterr/.cache/huggingface/datasets/csv/default-1cd582fe01726688/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

loading configuration file models/facebook_wav2vec2-large-960h-lv60-self_emotion_optimal_epochs_/checkpoint-1575/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "models/facebook_wav2vec2-large-960h-lv60-self_emotion_optimal_epochs_/checkpoint-1575",
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForSpeechClassification"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2

  0%|          | 0/52 [00:00<?, ?ba/s]

In [2]:
checkpoints*3

['facebook/wav2vec2-large-slavic-voxpopuli-v2',
 'facebook/wav2vec2-large-960h-lv60-self',
 'classla/wav2vec2-large-slavic-parlaspeech-hr',
 'facebook/wav2vec2-large-slavic-voxpopuli-v2',
 'facebook/wav2vec2-large-960h-lv60-self',
 'classla/wav2vec2-large-slavic-parlaspeech-hr',
 'facebook/wav2vec2-large-slavic-voxpopuli-v2',
 'facebook/wav2vec2-large-960h-lv60-self',
 'classla/wav2vec2-large-slavic-parlaspeech-hr']